#   8、使用蓝图的示例


In [4]:
#!/usr/bin/env  python
# -*-  coding:utf-8  -*-
import  xml.etree.cElementTree  as ET
from  collections  import  defaultdict
import  re

osm_file = open("chicago.osm", "r")

street_type_re = re.compile(r'\S+\.?$', re.IGNORECASE)
street_types = defaultdict(int)

def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if  m:
        street_type = m.group()
        
        street_types[street_type] += 1
        

def print_sorted_dict(d):
    keys = d.keys()
    keys = sorted(keys, key=lambda s: s.lower())
    for k in keys:
        v = d[k]
        print  "%s: %d"  % (k, v)
        
        
def is_street_name(elem):
    return (elem.tag == "tag") and (elem.attrib['k'] == "addr:street")


def  audit():
    for  event, elem  in  ET.iterparse(osm_file):
        if  is_street_name(elem):
            audit_street_type(street_types, elem.attrib['v'])
    print_sorted_dict(street_types)
    

if  __name__ == '__main__':
    audit()


IOError: [Errno 2] No such file or directory: 'chicago.osm'

# 11、审查交叉字段的约束条件 auditing  a  cross-field  constraint

In [5]:
def ensure_float(v):
    if is_number(v):
        return float(v)
    
def audit_population_density(input_file):
    for row in input_file:
        population = ensure_float(row['populationTotal'])
        area = ensure_float(row['areaLand'])
        population_desity = ensure_float(row['populationDensity'])
        if population and  area and population_desity:
            calculated_desity = population / area
            if math.fabs(calculated_desity - population_density) > 10:
                print "Possibly bad population density for ",row['name']
                
if  __name__ == '__main__':
    input_file = csv.DictReader(open("cities.csv"))
    skip_lines(input_file, 3)
    audit_population_density(input_file)

IOError: [Errno 2] No such file or directory: 'cities.csv'

# 12、修正有效性

In [8]:
"""
Your task is to check the "productionStartYear" of the DBPedia autos datafile for valid values.
The following things should be done:
- check if the field "productionStartYear" contains a year
- check if the year is in range 1886-2014
- convert the value of the field to be just a year (not full datetime)
- the rest of the fields and values should stay the same
- if the value of the field is a valid year in the range as described above,
  write that line to the output_good file
- if the value of the field is not a valid year as described above, 
  write that line to the output_bad file
- discard rows (neither write to good nor bad) if the URI is not from dbpedia.org
- you should use the provided way of reading and writing data (DictReader and DictWriter)
  They will take care of dealing with the header.

You can write helper functions for checking the data and writing the files, but we will call only the 
'process_file' with 3 arguments (inputfile, output_good, output_bad).
"""
import csv
import pprint
import pandas as pd

INPUT_FILE = 'autos.csv'
OUTPUT_GOOD = 'autos-valid.csv'
OUTPUT_BAD = 'FIXME-autos.csv'

def process_file(input_file, output_good, output_bad):
    data_good = []
    data_bad = []
    with open(input_file, "r") as f:
        reader = csv.DictReader(f)
        header = reader.fieldnames
        for  row  in  reader:
            if  row['URI'].find("dbpedia.org") < 0:
                continue
                
            ps_year = row['productionStartYear'][:4]
            try: # ues try/except  to filter  valid  items
                ps_year = int(ps_year)
                row['productionStartYear'] = ps_year
                if (ps_year >= 1886) and (ps_year <= 2014):
                    data_good.append(row)
                else:
                    data_bad.append(row)
            except  ValueError: # non-numeric strings caught by exception
                if ps_year == 'NULL':
                    data_bad.append(row)
 
    # This is just an example on how you can use csv.DictWriter
    # Remember that you have to output 2 files
    with open(output_good, "w") as good:
        writer = csv.DictWriter(good, delimiter=",", fieldnames= header)
        writer.writeheader()
        for row in data_good:
            writer.writerow(row)
            
    with open(output_bad, "w") as bad:
        writer = csv.DictWriter(bad, delimiter=',', fieldnames= header)
        writer.writeheader()
        for  row  in  data_bad:
            writer.writerow(row)


def test():

    process_file(INPUT_FILE, OUTPUT_GOOD, OUTPUT_BAD)


if __name__ == "__main__":
    test()

In [9]:
pd.read_csv('FIXME-autos.csv')['productionStartYear'].head()

0    2108.0
1       1.0
2       NaN
3       NaN
Name: productionStartYear, dtype: float64

In [10]:
import pandas as pd
pd.read_csv('autos-valid.csv')['productionStartYear'].head()

0    1989
1    1969
2    1957
3    1959
4    1936
Name: productionStartYear, dtype: int64

# 14/审查准确率 2

In [ ]:
client = MongoClient("mongodb://localhost:27017")
db = client.examples


def skip_lines(input_file, skip):
    for i in input_file:
        country = row['country_label']
        country = contry.strip()
        if (country == "NULL") or (country == ""):
            continue
        if db.countries.find({"name":country}).count() != 1:
            print "Not found:", country
            
if __name__ == '__main__':
    input_file = csv.

#  15、审查完整性

#  19/审查均匀性

In [11]:
import csv
import pprint

fieldname = "wgs84_pos#lat"
minval = -90
maxval = 90

def skip_lines(input_file, skip):
    for i in range(0, skip):
        next(input_file)
        
def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        return False
    
    
def audit_float_field(v, counts):
    v = v.strip()
    if v == "NULL":
        counts['nulls'] += 1
    elif  v == "":
        counts['empties'] += 1
    elif is_array(v):
        counts['arrays'] += 1
    elif not is_number(v):
        print "Found non number:", v
    else:
        v = float(v)
        if not ((minval < v) and (v < maxval)):
            print "Found out of range value:", v
            

if __name__ =="__main__":
    input_file = csv.DictReader(open("cities3.csv"))
    skip_lines(input_file, 3)
    counts = {"nulls": 0, "empties": 0 ,"arrays": 0}
    nrows = 0
    for row in input_file:
        audit_float_field(row[fieldname], counts)
        nvows += 1
        
    print "num cities:", nrows
    print "nulls:", counts['nulls']
    print "empties:", counts['empties']
    print "arrays:", counts['arrays']
    
        
        

IOError: [Errno 2] No such file or directory: 'cities3.csv'

# 课程6：习题集：数据质量

# 1、审核数据质量

In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
"""
In this problem set you work with cities infobox data, audit it, come up with a
cleaning idea and then clean it up. In the first exercise we want you to audit
the datatypes that can be found in some particular fields in the dataset.
The possible types of values can be:
- NoneType if the value is a string "NULL" or an empty string ""
- list, if the value starts with "{"
- int, if the value can be cast to int
- float, if the value can be cast to float, but CANNOT be cast to int.
   For example, '3.23e+07' should be considered a float because it can be cast
   as float but int('3.23e+07') will throw a ValueError
- 'str', for all other values

The audit_file function should return a dictionary containing fieldnames and a 
SET of the types that can be found in the field. e.g.
{"field1": set([type(float()), type(int()), type(str())]),
 "field2": set([type(str())]),
  ....
}
The type() function returns a type object describing the argument given to the 
function. You can also use examples of objects to create type objects, e.g.
type(1.1) for a float: see the test function below for examples.

Note that the first three rows (after the header row) in the cities.csv file
are not actual data points. The contents of these rows should note be included
when processing data types. Be sure to include functionality in your code to
skip over or detect these rows.
"""

In [13]:
import codecs
import csv
import json
import pprint

CITIES = 'cities.csv'

FIELDS = ["name","timeZone_label", "utcOffset","homepage", "governmentType_label", 
         "isPartOf_label", "areaCode", "populationTotal", "elevation", 
          "maximumElevation", "minimumElevation", "populationDensity",
         "wgs84_pos#lat", "wgs84_pos#long", "areaLand", "areaMetro", "areaUrban"]

def  skip_lines(input_file, skip):
    for i in range(skip):
        next(input_file)
        
        
def is_int(v):
    try:
        int(v)
        return True
    except ValueError:
        return False
    
def  is_float(v):
    try:
        float(v)
        return True
    except ValueError:
        return False

              

def audit_file(filename, fields):
    fieldtypes = {}
    for field in fields:
        fieldtypes[field] = set()
    with open(filename, "r") as f:
        reader = csv.DictReader(f)
        skip_lines(reader, 3)
        for row in reader:
            for field in fields:
                v = row[field]
                if v == "Null" or v == "":
                    fieldtypes[field].add(type(None))
                elif v.startswith('{'):
                    fieldtypes[field].add(type([]))
                elif is_int(v):
                    fieldtypes[field].add(type(1))
                elif is_float(v):
                    fieldtypes[field].add(type(1.1))
                else:
                    fieldtypes[field].add(type('1.1'))

    return fieldtypes


def test():
    fieldtypes = audit_file(CITIES, FIELDS)
    
    pprint.pprint(fieldtypes)
    
    assert fieldtypes["areaLand"] == set([type(1.1), type([]), type(None)])
    assert fieldtypes["areaMetro"] == set([type(1.1), type(None)])
    
if __name__ == "__main__":
    test()


{'areaCode': set([<type 'int'>, <type 'str'>]),
 'areaLand': set([<type 'float'>, <type 'list'>, <type 'str'>]),
 'areaMetro': set([<type 'float'>, <type 'str'>]),
 'areaUrban': set([<type 'float'>, <type 'str'>]),
 'elevation': set([<type 'float'>, <type 'list'>, <type 'str'>]),
 'governmentType_label': set([<type 'str'>]),
 'homepage': set([<type 'str'>]),
 'isPartOf_label': set([<type 'list'>, <type 'str'>]),
 'maximumElevation': set([<type 'str'>]),
 'minimumElevation': set([<type 'str'>]),
 'name': set([<type 'list'>, <type 'str'>]),
 'populationDensity': set([<type 'float'>, <type 'list'>, <type 'str'>]),
 'populationTotal': set([<type 'int'>, <type 'str'>]),
 'timeZone_label': set([<type 'str'>]),
 'utcOffset': set([<type 'int'>, <type 'list'>, <type 'str'>]),
 'wgs84_pos#lat': set([<type 'float'>]),
 'wgs84_pos#long': set([<type 'float'>])}


AssertionError: 

# 3、修复区域

In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
"""
In this problem set you work with cities infobox data, audit it, come up with a
cleaning idea and then clean it up.

Since in the previous quiz you made a decision on which value to keep for the
"areaLand" field, you now know what has to be done.

Finish the function fix_area(). It will receive a string as an input, and it
has to return a float representing the value of the area or None.
You have to change the function fix_area. You can use extra functions if you
like, but changes to process_file will not be taken into account.
The rest of the code is just an example on how this function can be used.
"""


In [ ]:
import codecs
import csv
import json
import pprint 

CITIES = 'cities.csv'

def fix_area(area):
    
    if area == "NULL":
        area = None
    elif is_float(area):
        area = float(area)
    else:
        area_parts = area.split("|")
        first_area = area_parts[0][1:]
        second_area = area_parts[1][:(len(area_parts[1]) - 1)]
        if len(first_area.split(".")[1]) > len(second_area.split(".")[1]):
            area = float(first_area)
        else:
            area = float(second_area)    
    
    return area

def is_float(s):
    try:
        float(s)
        return True
    except ValueError:
        return False



def process_file(filename):
    # change to this function will be ignored when you submit the exercise
    data = []
    
    with open(filename, "r") as f:
        reader = csv.DictReader(f)
        
        #skipping the extra metadata
        for i in range(3):
            l = reader.next()
            
        # processing file
        for line in  reader:
            # calling your function to fix the area value
            if "areaLand" in line:
                line["areaLand"] = fix_area(line["areaLand"])
            data.append(line)
            
    return data
            
def test():
    data = process_file(CITIES)
    
    print "Printing three example results:"
    for n in range(5, 8):
        pprint.pprint(data[n]["areaLand"])
        
    assert data[3]["areaLand"] == None
    assert data[8]["areaLand"] == 55166700.0
    assert data[20]["areaLand"] == 14581600.0
    assert data[33]["areaLand"] == 20564500.0
    
    
if __name__ == "__main__":
    test()


# 5/修复姓名

In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
"""
In this problem set you work with cities infobox data, audit it, come up with a
cleaning idea and then clean it up.

In the previous quiz you recognized that the "name" value can be an array (or
list in Python terms). It would make it easier to process and query the data
later if all values for the name are in a Python list, instead of being
just a string separated with special characters, like now.

Finish the function fix_name(). It will recieve a string as an input, and it
will return a list of all the names. If there is only one name, the list will
have only one item in it; if the name is "NULL", the list should be empty.
The rest of the code is just an example on how this function can be used.
"""

In [ ]:
import codecs
import csv
import pprint 

CITIES = "cities.csv"

def fix_name(name):
    if name == "NULL":
        name = []
    elif name.startswith("{"):
        name = name.replace("{","").replace("}","").split("|")
    else:
        name = [name] 

    return name


def process_file(filename):
    data = []
    with open(filename, "r") as f:
        reader = csv.DictReader(f)
        #skipping the extra metadata
        for i in range(3):
            l = reader.next()
        # processing file
        for line in reader:
            # calling your function to fix the area value
            if "name" in line:
                line["name"] = fix_name(line["name"])
            data.append(line)
    return data

def test():
    data = process_file(CITIES)
    
    print "Printing 20 results:"
    for n in range(20):
        pprint.pprint(data[n]["name"])
        
    assert data[14]["name"] == ["Negtemiut", "Nightmute"]
    assert data[9]["name"] == ['Pell City Alabama']
    assert data[3]["name"] == ['Kumhari']

if __name__ == "__main__":
    test()
    





# 6/交叉字段审查

In [14]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
"""
In this problem set you work with cities infobox data, audit it, come up with a
cleaning idea and then clean it up.

If you look at the full city data, you will notice that there are couple of
values that seem to provide the same information in different formats: "point"
seems to be the combination of "wgs84_pos#lat" and "wgs84_pos#long". However,
we do not know if that is the case and should check if they are equivalent.

Finish the function check_loc(). It will recieve 3 strings: first, the combined
value of "point" followed by the separate "wgs84_pos#" values. You have to
extract the lat and long values from the "point" argument and compare them to
the "wgs84_pos# values, returning True or False.

Note that you do not have to fix the values, only determine if they are
consistent. To fix them in this case you would need more information. Feel free
to discuss possible strategies for fixing this on the discussion forum.

The rest of the code is just an example on how this function can be used.
Changes to "process_file" function will not be taken into account for grading.
"""

'\nIn this problem set you work with cities infobox data, audit it, come up with a\ncleaning idea and then clean it up.\n\nIf you look at the full city data, you will notice that there are couple of\nvalues that seem to provide the same information in different formats: "point"\nseems to be the combination of "wgs84_pos#lat" and "wgs84_pos#long". However,\nwe do not know if that is the case and should check if they are equivalent.\n\nFinish the function check_loc(). It will recieve 3 strings: first, the combined\nvalue of "point" followed by the separate "wgs84_pos#" values. You have to\nextract the lat and long values from the "point" argument and compare them to\nthe "wgs84_pos# values, returning True or False.\n\nNote that you do not have to fix the values, only determine if they are\nconsistent. To fix them in this case you would need more information. Feel free\nto discuss possible strategies for fixing this on the discussion forum.\n\nThe rest of the code is just an example on ho

In [15]:
import csv
import pprint

CITIES = 'cities.csv'

def check_loc(point, lat, longi):
    pointLat, pointLongi = point.split(" ")[0], point.split(" ")[1]
    if pointLat == lat and pointLongi == longi:
        return True
    else:
        return False




def process_file(filename):
    data = []
    with open(filename,"r") as f:
        reader = csv.DictReader(f)
        # skipping the extra matadata
        for i in range(3):
            l = reader.next()
            
        # processing file
        for line in reader:
            # calling your function to check the location
            result = check_loce(line["point"], line["wgs94_pos#lat"], line["wgs84_pos#long"])
            if not result:
                print "{}:{} != {}{}".format(line["name"], line["point"], line["wgs84_pos#lat"], line["wgs84_pos#long"])
            data.append(line)
            
        return data


def test():
    assert check_loc("33.08 75.28", "33.08", "75.28") == True
    assert check_loc("44.57833333333333 -91.21833333333333", "44.5783", "-91.2183") == False
    
if __name__ == "__main__":
    test()


